In [17]:
import pandas as pd 
import json
from sklearn.feature_extraction.text import TfidfVectorizer

In [18]:
f = open('wishlist.json')
wishlist = json.load(f)
wishlist = pd.DataFrame(wishlist['products'])
wishlist 

,id,type,brand,model
0,hifuvovuosaci,CPU,AMD,Ryzen 9 5950X Processor
1,hifuvovuosaci,CPU,AMD,Ryzen 5 5600X Processor
2,hifuvovuosaci,CPU,AMD,Ryzen 7 5800X Processor


In [19]:
#for each product in the wishlist make request to api and get the
#products full description
f = open('wishlist-products.json')
wishlist_products = json.load(f)
wishlist_description = pd.DataFrame(wishlist_products['products']).pop('description')
wishlist_description = pd.DataFrame().from_records(wishlist_description)
wishlist_description

,# of CPU Cores,# of Threads,Base Clock,Max Boost Clock,Total L2 Cache,Total L3 Cache,Unlocked,CMOS,Package,PCI ExpressÂ® Version,Thermal Solution (PIB),Default TDP / TDP
0,16,32,3.4GHz,Up to 4.9GHz,8MB,64MB,Yes,TSMC 7nm FinFET,AM4,PCIe 4.0,Not included,105W
1,6,12,3.7GHz,Up to 4.6GHz,3MB,32MB,Yes,TSMC 7nm FinFET,AM4,PCIe 4.0,Wraith Stealth,65W
2,8,16,3.8GHz,Up to 4.7GHz,4MB,32MB,Yes,TSMC 7nm FinFET,AM4,PCIe 4.0,Not included,105W


In [20]:
#get all products in our database and their descriptions
f = open('products.json')
products = json.load(f)
products_description = pd.DataFrame(products['products']).pop('description')
products_description = pd.DataFrame().from_records(products_description)
products_description.columns.values

array(['# of CPU Cores', '# of Threads', 'Base Clock', 'Max Boost Clock',
       'Total L2 Cache', 'Total L3 Cache', 'Unlocked', 'CMOS', 'Package',
       'PCI ExpressÂ® Version', 'Thermal Solution (PIB)',
       'Default TDP / TDP', 'Processor Base Frequency',
       'Max Turbo Frequency', 'Cache', 'Bus Speed', 'TDP',
       'Configurable TDP-up Frequency', 'Configurable TDP-up',
       'Configurable TDP-down Frequency', 'Configurable TDP-down',
       'IntelÂ® Thermal Velocity Boost Frequency', '# of QPI Links',
       'IntelÂ® Turbo Boost Max Technology 3.0 Frequency',
       'IntelÂ® Turbo Boost Technology 2.0 Frequency', '# of GPU Cores',
       'cTDP'], dtype=object)

In [33]:
df_products_des = products_description[['# of CPU Cores', '# of Threads', 'Base Clock', 'Max Boost Clock',
       'Total L2 Cache', 'Total L3 Cache', 'Default TDP / TDP', 'Processor Base Frequency',
       'Max Turbo Frequency', 'Cache', 'TDP']]
df_products_des

,# of CPU Cores,# of Threads,Base Clock,Max Boost Clock,Total L2 Cache,Total L3 Cache,Default TDP / TDP,Processor Base Frequency,Max Turbo Frequency,Cache,TDP
0,16,32,3.4GHz,Up to 4.9GHz,8MB,64MB,105W,NaN,NaN,NaN,NaN
1,6,12,NaN,NaN,NaN,NaN,NaN,1.10GHz,4.70GHz,12 MB IntelÂ® Smart Cache,15 W
2,4,8,NaN,NaN,NaN,NaN,NaN,4.30GHz,4.50GHz,8 MB,112 W
3,8,16,NaN,NaN,NaN,NaN,NaN,3.5GHz,5.3GHz,16 MB IntelÂ® Smart Cache,125 W
4,8,16,3.8GHz,Up to 4.6GHz,4MB,16MB,65W,NaN,NaN,NaN,NaN
5,12,24,3.8GHz,Up to 4.7GHz,6MB,64MB,105W,NaN,NaN,NaN,NaN
6,8,16,3.9GHz,Up to 4.7GHz,4MB,32MB,105W,NaN,NaN,NaN,NaN
7,16,32,3.5GHz,Up to 4.4GHz,8MB,32MB,180W,NaN,NaN,NaN,NaN
8,6,12,3.7GHz,Up to 4.6GHz,3MB,32MB,65W,NaN,NaN,NaN,NaN
9,8,16,3.8GHz,Up to 4.7GHz,4MB,32MB,105W,NaN,NaN,NaN,NaN


In [38]:
df_change = df_products_des.loc[df_products_des['Base Clock'].isnull()]
df_change.drop(['Base Clock', 'Default TDP / TDP', 'Max Boost Clock'], axis=1, inplace=True)
df_change
#df_products_des

C:\Users\lmoto\anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,# of CPU Cores,# of Threads,Total L2 Cache,Total L3 Cache,Processor Base Frequency,Max Turbo Frequency,Cache,TDP
1,6,12,NaN,NaN,1.10GHz,4.70GHz,12 MB IntelÂ® Smart Cache,15 W
2,4,8,NaN,NaN,4.30GHz,4.50GHz,8 MB,112 W
3,8,16,NaN,NaN,3.5GHz,5.3GHz,16 MB IntelÂ® Smart Cache,125 W


In [39]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(ds['description'])

NameError: name 'ds' is not defined